In [1]:
import argparse
import os
import math
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import random
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import torchvision.datasets as dset
import matplotlib.pyplot as plt
# from torchsummary import summary
#os.chdir('D:/GANart/GAN-Artwork-Generation')
import time
from PIL import Image

In [ ]:
import torch
import random
import matplotlib.pyplot as plt
from torch.autograd import Variable

def guassian_kernel(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    """
    source: sample_size_1 * feature_size original dataset 
    target: sample_size_2 * feature_size target dataset
    kernel_mul: caculate every kernel bandwith
    kernel_num: kernel_num
    fix_sigma: fix_sigma 

		return: (sample_size_1 + sample_size_2) * (sample_size_1 + sample_size_2)的
						matrix:
						[	K_ss K_st
							K_ts K_tt ]
    """
    n_samples = int(source.size()[0])+int(target.size()[0])
    total = torch.cat([source, target], dim=0) # cat function
    
    total0 = total.unsqueeze(0).expand(int(total.size(0)), \ # 5 dimensions (1,batchsize,channel, width, height) 
                                       int(total.size(0)), \ # single pictures only need 4 dimension 
                                       int(total.size(1)),
                                    int(total.size(2)),
                                      int(total.size(3)))
    total1 = total.unsqueeze(1).expand(int(total.size(0)), \
                                       int(total.size(0)), \
                                       int(total.size(1)),
                                    int(total.size(2)),
                                      int(total.size(3)))
    L2_distance = ((total0-total1)**2).sum(4) # 计算高斯核中的|x-y|  #using guassian kernle to caculate 
#     print(L2_distance)
    
    # caculate every kernel bandwidth 
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
    
    # exp(-|x-y|/bandwith)
    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for \
                  bandwidth_temp in bandwidth_list]

    return sum(kernel_val) 
  
def mmd(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    batch_size = int(source.size()[0])
    kernels = guassian_kernel(source, target,
                              kernel_mul=kernel_mul, 	
                             	kernel_num=kernel_num, 	
                              fix_sigma=fix_sigma)
#     print(kernels.shape)
#     print(batch_size)
    XX = kernels[:batch_size, :batch_size] # Source<->Source
    YY = kernels[batch_size:, batch_size:] # Target<->Target
    XY = kernels[:batch_size, batch_size:] # Source<->Target
    YX = kernels[batch_size:, :batch_size] # Target<->Source
    loss = torch.mean(XX + YY - XY -YX) 

    return loss


if __name__ == "__main__":
    import numpy as np
    data_1 = torch.tensor(np.random.normal(0,10,(100,50)))
    data_2 = torch.tensor(np.random.normal(10,10,(100,50)))

#     print("MMD Loss:",mmd(data_1,data_2))

    data_1 = torch.tensor(np.random.normal(0,10,(100,50)))
    data_2 = torch.tensor(np.random.normal(0,9,(100,50)))

#     print("MMD Loss:",mmd(data_1,data_2))



In [4]:
# Root directory for dataset
dataroot = "../gan-fl/bestfake" #data path

# Number of workers for dataloader
workers = 1


# Batch size during training
batch_size = 4

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 256

#number of calsses lable 
n_class = 1

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator （output)
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epoc50
num_epochs = 180

# Learning rate for optimizers
lr = 0.0018


# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 2

In [5]:
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

In [6]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
savelist1=[]
for epoch in range(1):
    # For each batch in the dataloader 
    for i, (data, real_style_labels) in enumerate(dataloader, 0):
        

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        # Format batch
        
        real_cpu = data.to(device)
        if i<202:
            savelist1.append(real_cpu)

NameError: name 'dataloader' is not defined

In [ ]:
mmdli=[]
for i in range(200):
    mmdloss=mmd(savelist[i],savelist1[i]) # original list and generate list
    print(mmdloss)
    mmdli.append(mmdloss.cpu().numpy())
print('mean:',np.mean(mmdli))
print('minimum:',np.min(mmdli))
print('max:',np.max(mmdli))

In [8]:
# Root directory for dataset
dataroot2 = "C:/Users/Jackie/Desktop/science/oct/disease6511" #original dataset root

# Number of workers for dataloader
workers = 1


# Batch size during training
batch_size = 4

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 256

#number of calsses lable 
n_class = 1

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator （output)
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epoc50
num_epochs = 180

# Learning rate for optimizers
lr = 0.0018


# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 2

In [9]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
dataset2 = dset.ImageFolder(root=dataroot2,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader2 = torch.utils.data.DataLoader(dataset2, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

In [10]:
savelist=[]
for epoch in range(1):
    # For each batch in the dataloader 
    for i, (data, real_style_labels) in enumerate(dataloader2, 0):


        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        # Format batch
        
        real_cpu2 = data.to(device)
        if i<202:
#             print(real_cpu2)
            savelist.append(real_cpu2)

In [11]:
mmdli=[]
for i in range(200):
    mmdloss=mmd(savelist[i],savelist1[i])
    print(mmdloss)
    mmdli.append(mmdloss.cpu().numpy())
print('mean:',np.mean(mmdli))
print('minimum:',np.min(mmdli))
print('max:',np.max(mmdli))

NameError: name 'mmd' is not defined

In [38]:
# Root directory for dataset
dataroot10 = "../gan-fl/netG-oct-4E-4000-fourth-1000-1-50-fake" #generate dataset

# Number of workers for dataloader
workers = 1


# Batch size during training
batch_size = 4

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 256

#number of calsses lable 
n_class = 1

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator （output)
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epoc50
num_epochs = 180

# Learning rate for optimizers
lr = 0.0018


# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 2

In [39]:
dataset10 = dset.ImageFolder(root=dataroot10,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader10 = torch.utils.data.DataLoader(dataset10, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)
savelist10=[]
for epoch in range(1):
    # For each batch in the dataloader 
    for i, (data, real_style_labels) in enumerate(dataloader10, 0):
    
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        # Format batch
        
        real_cpu10 = data.to(device)
        if i<202:
#             print(real_cpu2)
            savelist10.append(real_cpu10)

In [40]:
mmdli=[]
for i in range(200):
    mmdloss=mmd(savelist[i],savelist10[i])
    print(mmdloss)
    mmdli.append(mmdloss.cpu().numpy())
print('mean:',np.mean(mmdli))
print('minimum:',np.min(mmdli))
print('max:',np.max(mmdli))

tensor(0.0050, device='cuda:0')
tensor(0.0041, device='cuda:0')
tensor(0.0068, device='cuda:0')
tensor(0.0058, device='cuda:0')
tensor(0.0045, device='cuda:0')
tensor(0.0115, device='cuda:0')
tensor(0.0089, device='cuda:0')
tensor(0.0050, device='cuda:0')
tensor(0.0043, device='cuda:0')
tensor(0.0036, device='cuda:0')
tensor(0.0041, device='cuda:0')
tensor(0.0082, device='cuda:0')
tensor(0.0048, device='cuda:0')
tensor(0.0072, device='cuda:0')
tensor(0.0075, device='cuda:0')
tensor(0.0087, device='cuda:0')
tensor(0.0114, device='cuda:0')
tensor(0.0047, device='cuda:0')
tensor(0.0041, device='cuda:0')
tensor(0.0053, device='cuda:0')
tensor(0.0067, device='cuda:0')
tensor(0.0134, device='cuda:0')
tensor(0.0100, device='cuda:0')
tensor(0.0076, device='cuda:0')
tensor(0.0053, device='cuda:0')
tensor(0.0076, device='cuda:0')
tensor(0.0069, device='cuda:0')
tensor(0.0043, device='cuda:0')
tensor(0.0053, device='cuda:0')
tensor(0.0061, device='cuda:0')
tensor(0.0031, device='cuda:0')
tensor(0